# Sentiment Analysis - Amazon baby data

### Load the data and remove NAN

In [2]:
import pandas as pd
products = pd.read_csv(f"D:/SYED/data/baby/amazon_baby.csv")
products = products.fillna({'review':''})  # fill in N/A's in the review column
products['review'].isnull().values.any()
products.shape

(183531, 3)

## Remove punctuation marks

In [3]:
import string
products['review_clean'] = products['review'].apply(lambda x: x.translate(str.maketrans('', '', 
                           string.punctuation)))
products.head(3)

,name,review,rating,review_clean
0,Planetwise Flannel Wipes,"These flannel wipes are OK, but in my opinion ...",3,These flannel wipes are OK but in my opinion n...
1,Planetwise Wipe Pouch,it came early and was not disappointed. i love...,5,it came early and was not disappointed i love ...
2,Annas Dream Full Quilt with 2 Shams,Very soft and comfortable and warmer than it l...,5,Very soft and comfortable and warmer than it l...


### Remove rating of 3

In [4]:
products = products[products['rating'] != 3]
products.head(3)

,name,review,rating,review_clean
1,Planetwise Wipe Pouch,it came early and was not disappointed. i love...,5,it came early and was not disappointed i love ...
2,Annas Dream Full Quilt with 2 Shams,Very soft and comfortable and warmer than it l...,5,Very soft and comfortable and warmer than it l...
3,Stop Pacifier Sucking without tears with Thumb...,This is a product well worth the purchase. I ...,5,This is a product well worth the purchase I h...


### More than 3 is +1 else -1

In [5]:
products['sentiment'] = products['rating'].apply(lambda rating : +1 if rating > 3 else -1)
products['review'][27401]

'I love this product.Simple but does the job great.Very easy to attach.I really have nothing bad to say about it.My baby is now protected from the sun.'

## Load the data as test and train
* `train-idx.json` for Training data
* `test-idx.json` for Test data

train_data_indices = pd.read_json(f"D:/repos/CourseraPlus/1_ML_Combined_Courses/Course/Week-2/train-idx.json")
train_data = pd.DataFrame(products, index = train_data_indices[0])
test_data_indices = pd.read_json(f"D:/repos/CourseraPlus/1_ML_Combined_Courses/Course/Week-2/test-idx.json")
test_data = pd.DataFrame(products, index = test_data_indices[0])
print("Train set: " + str(train_data.shape))
print("Test set: " + str(test_data.shape))

In [15]:
# To generate the test index list
test_index = pd.read_json('test-idx.json', orient='values')
test_index_list = [x[0] for x in test_index.values]
train_data = products.copy()
train_data = train_data.drop(train_data.index[test_index_list])

# Train index
train_index = pd.read_json('train-idx.json', orient='values')
train_index_list = [x[0] for x in train_index.values]
test_data = products.copy()
test_data = test_data.drop(test_data.index[train_index_list])

### Remove all the NaN before carrying out training

In [16]:
train_data.dropna(subset = ["name"], inplace=True)
train_data.isnull().values.sum()
test_data.dropna(subset = ["name"], inplace=True)
test_data.isnull().values.sum()
print("Train set: " + str(train_data.shape))
print("Test set: " + str(test_data.shape))

Train set: (133174, 5)
Test set: (33282, 5)


## Dictionary of word counts

In [17]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(token_pattern = r'\b\w+\b')
train_matrix = vectorizer.fit_transform(train_data['review_clean'])
test_matrix = vectorizer.transform(test_data['review_clean'])
print("Train matrix: " + str(train_matrix.shape))
print("Test matrix: " + str(test_matrix.shape))

Train matrix: (133174, 121539)
Test matrix: (33282, 121539)


## Logistic Regression Model

In [18]:
from sklearn.linear_model import LogisticRegression
sentiment_model = LogisticRegression(random_state=0).fit(train_matrix, train_data['sentiment'])
zero_elem = (sentiment_model.coef_ >= 0).sum()
less_elem = (sentiment_model.coef_ < 0).sum()
total_val = zero_elem + less_elem
print("Co-efficients with value more than 0: " + str(zero_elem) + " out of " + str(total_val))

Co-efficients with value more than 0: 89406 out of 121539


c:\Users\SYED\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [20]:
sample_test_data = test_data[10:13]
sample_test_data

,name,review,rating,review_clean,sentiment
59,Our Baby Girl Memory Book,Absolutely love it and all of the Scripture in...,5,Absolutely love it and all of the Scripture in...,1
71,Wall Decor Removable Decal Sticker - Colorful ...,Would not purchase again or recommend. The dec...,2,Would not purchase again or recommend The deca...,-1
91,New Style Trailing Cherry Blossom Tree Decal R...,Was so excited to get this product for my baby...,1,Was so excited to get this product for my baby...,-1


In [21]:
sample_test_matrix = vectorizer.transform(sample_test_data['review_clean'])
scores_sample = sentiment_model.decision_function(sample_test_matrix)
print(scores_sample)

[  5.12410325  -3.07075112 -10.32762058]


### Mapping function based on scores

In [22]:
def check_score(input_array):
    result = []
    for x in input_array:
        if x >= 0:
            result.append(1)
        else:
            result.append(-1)
    return result
check_score(scores_sample)

[1, -1, -1]

### Check probabilities

In [23]:
prob_val = sentiment_model.predict_proba(sample_test_matrix)
print(prob_val[:,1])

[9.94083659e-01 4.43299954e-02 3.27157711e-05]


## Full data sentiment analysis

In [24]:
#predictions = sentiment_model.predict(test_matrix)
final_scores_data = vectorizer.transform(test_data['review_clean'])
final_score = sentiment_model.decision_function(final_scores_data)
df = pd.DataFrame(final_score, columns = ['Score'])
print(df.shape)
print(df.columns)

(33282, 1)
Index(['Score'], dtype='object')


In [25]:
df_test = test_data
df_test.reset_index(drop = True, inplace = True)
df2 = pd.concat([df, df_test], axis = 1)
df_export = df2.sort_values(by='Score', ascending=False)
df_export.head(5)

,Score,name,review,rating,review_clean,sentiment
18081,50.916705,"Infantino Wrap and Tie Baby Carrier, Black Blu...",I bought this carrier when my daughter was abo...,5,I bought this carrier when my daughter was abo...,1
15702,46.098226,Baby Einstein Around The World Discovery Center,I am so HAPPY I brought this item for my 7 mon...,5,I am so HAPPY I brought this item for my 7 mon...,1
30582,44.390333,Graco FastAction Fold Jogger Click Connect Str...,Graco's FastAction Jogging Stroller definitely...,5,Gracos FastAction Jogging Stroller definitely ...,1
21498,41.136815,Roan Rocco Classic Pram Stroller 2-in-1 with B...,Great Pram Rocco!!!!!!I bought this pram from ...,5,Great Pram RoccoI bought this pram from Europe...,1
24250,40.206646,"Britax 2012 B-Agile Stroller, Red",[I got this stroller for my daughter prior to ...,4,I got this stroller for my daughter prior to t...,1


## Export data (Top & Bot 20)

In [26]:
df_top = df_export.head(20)
df_top.to_csv('top20.csv')
df_bottom = df_export.tail(20)
df_bottom.to_csv('bottom20.csv')

## Calculate Accuracy - Sentiment Model

In [27]:
print("Scores dataframe shape: " + str(df.shape))
print("Test data sentiment as reference shape: " + str(test_data.shape))
accuracy_sentiment_model = sentiment_model.score(final_scores_data, test_data['sentiment'].values)
print("Sentiment model accuracy:\n" + str(accuracy_sentiment_model))

Scores dataframe shape: (33282, 1)
Test data sentiment as reference shape: (33282, 5)
Sentiment model accuracy:
0.9329968150952467


## Special Classifier

In [28]:
significant_words = ['love', 'great', 'easy', 'old', 'little', 'perfect', 'loves', 
      'well', 'able', 'car', 'broke', 'less', 'even', 'waste', 'disappointed', 
      'work', 'product', 'money', 'would', 'return']

In [29]:
vectorizer_word_subset = CountVectorizer(vocabulary = significant_words) # limit to 20 words
train_matrix_word_subset = vectorizer_word_subset.fit_transform(train_data['review_clean'])
test_matrix_word_subset = vectorizer_word_subset.transform(test_data['review_clean'])
print("Train matrix special: " + str(train_matrix_word_subset.shape))
print("Test matrix special: " + str(test_matrix_word_subset.shape))

Train matrix special: (133174, 20)
Test matrix special: (33282, 20)


## Simple Log-Reg model

In [30]:
simple_model = LogisticRegression(random_state=0).fit(train_matrix_word_subset, train_data['sentiment'])
zero_elem_simple = (simple_model.coef_ > 0).sum()
print(zero_elem_simple)

10


### Co-efficient dictionary for significant words

In [31]:
arr1 = simple_model.coef_
arr2 = sentiment_model.coef_
arr1 = arr1.reshape(-1, 1)
arr2 = arr2.reshape(-1, 1)
list1 = arr1.tolist()
list2 = arr2.tolist()
vectorizer.get_feature_names_out()

# Check the coefficients for 20 words used in model
output_dict = dict(zip(significant_words, list1))
setiment_dict = dict(zip(vectorizer.get_feature_names_out(), list2))
#print(setiment_dict)

## Compare two dictionaries

In [32]:
def compare_ndic(src, dest):
    result = []
    for skey, sval in src.items():
        for nkey, nval in dest.items():
            if skey == nkey:
                result.append([skey, sval, nval])
    return result

new_list = compare_ndic(output_dict, setiment_dict)
df_dict = pd.DataFrame(new_list, columns=['Word', 'Small', 'Sentiment'])
df_dict.head(20)

,Word,Small,Sentiment
0,love,[1.364698227557282],[1.5653991227979211]
1,great,[0.94485795953734],[1.2442616552202135]
2,easy,[1.188593764857798],[1.3687686958571474]
3,old,[0.08474509132707782],[0.02447795134998793]
4,little,[0.5188840456186552],[0.5141651289615018]
5,perfect,[1.5090928051684753],[1.96956698245392]
6,loves,[1.673652147725519],[1.6821186959298344]
7,well,[0.5038990477323433],[0.481462521353968]
8,able,[0.19069309989471106],[0.32587129016290445]
9,car,[0.058886981389063925],[0.10092833643702658]


# Compare `train` & `test` accuracy

In [33]:
accuracy_sentiment_model = sentiment_model.score(final_scores_data, test_data['sentiment'].values)
print("Sentiment model accuracy TRAINING data:\n" + str(accuracy_sentiment_model))

accuracy_simple_model = simple_model.score(test_matrix_word_subset, test_data['sentiment'].values)
print("Simple model accuracy TRAINING data:\n" + str(accuracy_simple_model))

Sentiment model accuracy TRAINING data:
0.9329968150952467
Simple model accuracy TRAINING data:
0.8692987200288445


In [36]:
test_data['sentiment'].value_counts()

 1    28048
-1     5234
Name: sentiment, dtype: int64

In [37]:
print("Majority classifier accuracy should be:\n" + str(28048 /(28048 + 5234)))

Majority classifier accuracy should be:
0.8427378162370049
